# FutureHouse platform client usage example

In [ ]:
import time

from crow_client import CrowClient, JobNames
from crow_client.models import (
    AuthType,
    JobRequest,
    RuntimeConfig,
    Stage,
)
from ldp.agent import AgentConfig

## Client instantiation

Here we use `auth_type=AuthType.API_KEY` to authenticate with the platform.
Please log in to the platform and go to your user settings to get your API key.

In [ ]:
client = CrowClient(
    stage=Stage.DEV,
    auth_type=AuthType.API_KEY,
    api_key="your-api-key",
)

## Submit a job

Submitting jobs is done by calling the `create_job` method, which receives a `JobRequest` object.

In [ ]:
job_data = JobRequest(
    name=JobNames.from_string("crow"),
    query="What is the molecule known to have the smallest solubility in water?",
)
client.create_job(job_data)

while client.get_job().status != "success":
    time.sleep(5)
print(f"Job status: {client.get_job().status}")
print(f"Job answer: \n{client.get_job().formatted_answer}")

Job status: success
Job answer: 
Question: What is the molecule known to have the smallest solubility in water?

Based on the available excerpts, the molecule reported to have the smallest solubility in water is 2,6,10,14‐tetramethylpentadecane. This conclusion is derived from quantitative measurements in the study by Tolls et al., which focused on the aqueous solubility of C10‐ to C19‐alkanes.

Tolls et al. measured solubility values using advanced techniques (including headspace solid-phase microextraction and purge and trap/thermal desorption) to determine extremely low concentrations. In their study, they report that among a range of saturated hydrocarbons, 2,6,10,14‐tetramethylpentadecane shows the lowest solubility, with a value of 0.01 μg/L. This corresponds to a molar concentration of approximately 3.7 × 10⁻¹¹ M (tolls2002aqueoussolubility−molecularsize pages 3-4). In the same study, pentadecane is reported with a solubility of 0.04 μg/L (1.9 × 10⁻¹⁰ M), which is notably higher

You can also pass a `runtime_config` to the job, which will be used to configure the agent on runtime.
Here, we will define a agent configuration and pass it to the job. This agent is used to decide the next action to take.
We will also use the `max_steps` parameter to limit the number of steps the agent will take.

In [ ]:
agent = AgentConfig(
    agent_type="SimpleAgent",
    agent_kwargs={
        "model": "gpt-4o",
        "temperature": 0.0,
    },
)
job_data = JobRequest(
    name=JobNames.CROW,
    query="How many moons does earth have?",
    runtime_config=RuntimeConfig(agent=agent, max_steps=10),
)
client.create_job(job_data)

while client.get_job().status != "success":
    time.sleep(5)
print(f"Job status: {client.get_job().status}")
print(f"Job answer: \n{client.get_job().formatted_answer}")

Job status: success
Job answer: 
Question: How many moons does earth have?

Earth has one permanent natural satellite—the Moon—which has been Earth’s constant companion for over four billion years (jedicke2018earthsminimoonsopportunities pages 1-4; canup2012formingamoon pages 1-3; moonsUnknownyearallmoonsgreat pages 1-2). While Earth’s gravitational field sometimes captures additional small objects, these “minimoons” are temporary natural satellites, not permanent additions to the system (jedicke2018earthsminimoonsopportunities pages 4-6; jedicke2018earthsminimoonsopportunities pages 16-18).

The permanent Moon, with a diameter on the order of ~3,500 km, is the outcome of a singular, giant impact event—a collision between proto-Earth and a Mars-sized body—which produced a circumterrestrial debris disk that eventually coalesced into today’s Moon (canup2012formingamoon pages 1-3; belbruno2005wheredidthe pages 45-48). This formation process not only explains the Moon’s bulk properties and

# Continue a job

The platform allows to keep asking follow-up questions to the previous job.
To accomplish that, we can use the `runtime_config` to pass the `job_id` of the previous job.

Notice that `create_job` accepts both a `JobRequest` object and a dictionary with keywords arguments.

In [ ]:
job_data = JobRequest(
    name=JobNames.CROW,
    query="How many species of birds are there?"
)

job_id = client.create_job(job_data)
while client.get_job().status != "success":
    time.sleep(5)
print(f"First job status: {client.get_job().status}")
print(f"First job answer: \n{client.get_job().formatted_answer}")

First job status: success
First job answer: 
Question: How many species of birds are there?

There is no single answer because the count depends on the species concept and taxonomic approach used. Traditional checklists based on the biological species concept recognize roughly 9,000–11,000 species. For example, traditional lists such as those referenced by Barrowclough et al. list approximately 9,159 biological species (which later revisions brought up to around 10,021), and other authoritative treatments like the IOC World Bird List report counts around 10,828 species (isaksson2018impactofurbanization pages 15–18; isaksson2018impactofurbanization pages 1–8). In contrast, when applying a phylogenetic or evolutionary species concept—which splits biological species into smaller, diagnosable evolutionary units—the total number increases substantially.

Barrowclough et al. (2016) performed a morphological study by examining a random sample of 200 biological species and found that on averag

In [ ]:
continued_job_data = {
    "name": JobNames.CROW,
    "query": "From the previous answer, specifically,how many species of crows are there?",
    "runtime_config": {
        "continued_job_id": job_id
    }
}

continued_job_id = client.create_job(continued_job_data)
while client.get_job().status != "success":
    time.sleep(5)
print(f"Continued job status: {client.get_job().status}")
print(f"Continued job answer: \n{client.get_job().formatted_answer}")

Continued job status: success
Continued job answer: 
Question: You are following up on a prior task: How many species of birds are there?

To perform a new task using prior context: From the previous answer, specifically,how many species of crows are there?

Crows, defined here as members of the genus Corvus, are estimated to comprise approximately 40 species worldwide (krol2020crowsandravens pages 1-3). This number is derived from an analysis of the Corvidae family, where crows constitute around one‐third of the overall diversity. The referenced study explicitly states that these 40 crow species are distributed across all continents except South America and Antarctica, as well as in certain remote archipelagos such as Hawaii, Micronesia, and Melanesia (krol2020crowsandravens pages 1-3).

It is important to note that while other sources in the context address the broader family Corvidae or focus on urban species of corvids, none of these sources contradict the specific figure provided 